# Test Random Environment with Gym

In [1]:
import gym
import random
import pygame

In [28]:
env = gym.make('CartPole-v1')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [29]:
episodes = 10

for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0,1])
        _, reward, done,*_  = env.step(action)
        score += reward
        env.render()
    print(f'Episode: {episode} Score: {score}')
env.close()

Episode: 1 Score: 27.0
Episode: 2 Score: 21.0
Episode: 3 Score: 23.0
Episode: 4 Score: 20.0
Episode: 5 Score: 17.0
Episode: 6 Score: 11.0
Episode: 7 Score: 21.0
Episode: 8 Score: 28.0
Episode: 9 Score: 27.0
Episode: 10 Score: 17.0


/opt/homebrew/anaconda3/lib/python3.11/site-packages/gym/envs/classic_control/cartpole.py:211: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(
/opt/homebrew/anaconda3/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


# Create a DeepLearning model with Keras

In [30]:
import numpy as np

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

In [32]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape = (1, states)))
    model.add(Dense(24, activation= 'relu'))
    model.add(Dense(24, activation= 'relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [33]:
import tensorflow as tf
from keras import __version__
tf.keras.__version__ = __version__

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [34]:
model = build_model(states, actions)

In [35]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 4)                 0         
                                                                 
 dense_6 (Dense)             (None, 24)                120       
                                                                 
 dense_7 (Dense)             (None, 24)                600       
                                                                 
 dense_8 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770 (3.01 KB)
Trainable params: 770 (3.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Build Agent with keras RL

In [36]:
from tensorflow.keras.optimizers.legacy import Adam

In [37]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit = 50000, window_length = 1)
    dqn = DQNAgent(model = model, memory= memory, policy= policy,
                  nb_actions = actions, nb_steps_warmup= 10, target_model_update=0.01)
    return dqn

In [38]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=0.003), metrics=['mae'])
dqn.fit(env, nb_steps = 50000, visualize=False, verbose=1)

2024-01-22 01:29:11.671637: W tensorflow/c/c_api.cc:305] Operation '{name:'dense_8/bias/Assign' id:1230 op device:{requested: '', assigned: ''} def:{{{node dense_8/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_8/bias, dense_8/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Training for 50000 steps ...
Interval 1 (0 steps performed)


ValueError: Error when checking input: expected flatten_2_input to have shape (1, 4) but got array with shape (1, 2)